## 1. Setup & Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import tsforge as tsf
# Nixtla utilities
from utilsforecast.preprocessing import fill_gaps

# Configuration
pd.set_option("display.max_rows", 10)
pd.set_option("display.width", 120)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Paths
DATA_DIR = Path('./data')
OUTPUT_DIR = Path('./output')
OUTPUT_DIR.mkdir(exist_ok=True)

print("✓ Setup complete")

ModuleNotFoundError: No module named 'tsforge'

In [1]:
# read in our tsfeatures from last module 
df = pd.read_parquet(
    "/Users/jackrodenberg/Desktop/real-world-forecasting-foundations/output/m5_clean.parquet"
)

df.head()

NameError: name 'pd' is not defined

In [6]:
# now lets add our hierarchy information back in, to do that is a simple m:1 join as we load m5 data 

full_df = tsf.load_m5(data_dir=DATA_DIR,include_hierarchy=True,create_unique_id=True)

LOADING M5 DATA
✓ M5 cache detected. Loading from local files...
✓ Loaded in 0.7s
  Shape: 47,649,940 rows × 3 columns
  Memory: 638.1 MB
  Columns: unique_id, ds, y
  Returning: Y_df, X_df, S_df (all 3 dataframes)

🏗️ Expanding hierarchy...
  Expanding hierarchy via S_df merge...
  ✓ Added hierarchy columns: ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

🔑 Creating unique_id...
Created unique_id: 30,490 unique series
Sample: FOODS_1_001_CA_1

LOAD COMPLETE
  Shape: 47,649,940 rows × 8 columns
  Columns: ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'ds', 'y', 'unique_id']
  Applied: hierarchy expanded, unique_id created


In [9]:
static_df = full_df.select_dtypes(include=['object','category']).drop_duplicates()

# merge datasets 
df_feats = (
    df.merge(
        static_df,
        on=['unique_id'],
        how='left',
        validate='1:1'
    )
)

df_feats.head()

,unique_id,adi,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,...,p75,p95,p97point5,max,min,item_id,dept_id,cat_id,store_id,state_id
0,FOODS_1_001_CA_1,1.0,1,52,0.202104,0.000013,-2.381430,2.264020,0.344401,0.318924,...,0.605521,1.601187,2.347938,4.090354,-1.385813,FOODS_1_001,FOODS_1,FOODS,CA_1,CA
1,FOODS_1_001_CA_2,1.0,1,52,0.199255,0.000014,-2.632977,0.886323,0.223670,0.257528,...,0.491251,1.681104,2.186791,4.910704,-1.378517,FOODS_1_001,FOODS_1,FOODS,CA_2,CA
2,FOODS_1_001_CA_3,1.0,1,52,0.181853,0.000008,-2.645001,1.313087,0.120079,0.076005,...,0.720388,1.815111,2.280368,3.379001,-1.312669,FOODS_1_001,FOODS_1,FOODS,CA_3,CA
3,FOODS_1_001_CA_4,1.0,1,52,0.096339,0.000006,-2.199347,1.740567,0.144502,0.094568,...,0.703854,2.135249,2.135249,3.089512,-1.204673,FOODS_1_001,FOODS_1,FOODS,CA_4,CA
4,FOODS_1_001_TX_1,1.0,1,52,0.243264,0.000012,-0.718647,-0.456358,0.111054,0.062631,...,0.493887,2.058788,2.573902,5.449407,-1.071014,FOODS_1_001,FOODS_1,FOODS,TX_1,TX
